In [45]:
# recalc_opts.py
def recalc_opts(ib, df):
    '''Recalculates option prices and margins
    Arg:
       (ib) as connection object
       (df) as the dataframe of options
    Returns: dataframe with updated OptPrices and OptMargins
    '''
    opt_contracts = [Contract(conId=c) for c in df.optId]
    qopts = ib.qualifyContracts(*opt_contracts)
    tickers = ib.reqTickers(*qopts)
    optPrices = {t.contract.conId: t.marketPrice() for t in tickers} # {symbol: undPrice}
    optMargins = {c.conId: ib.whatIfOrder(c,Order(action='SELL', orderType='MKT', totalQuantity=abs(ls), whatIf=True)).initMarginChange 
        for c, ls in (zip(qopts, df.lotsize))}
    
    # update prices and margins
    df = df.set_index('optId')
    df.optPrice.update(pd.Series(optPrices))
    df.optMargin.update(pd.Series(optMargins))
    df.optMargin = df.optMargin.astype('float')
    df = df.assign(rom=df.optPrice*df.lotsize/df.optMargin*252/df.dte)
    
    return df.reset_index()

In [2]:
# get the remaining quantities
from ib_insync import *
util.startLoop()
from helper import get_nse_remqty, get_connected
with get_connected('nse', 'live') as ib:
    remqty = get_nse_remqty(ib)

Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests


In [12]:
from os import listdir
import pandas as pd

from helper import grp_opts, get_prec, recalc_opts

fspath = '../data/nse/'  # path for nse pickles

minOptPrice = 0.15
minRom = 0.1

nLargest = 5

fs = listdir(fspath)

cols = ['optId', 'symbol', 'right', 'expiration', 'dte', 'strike', 'undPrice', 
'lo52', 'hi52', 'Fall', 'Rise', 'loFall', 'hiRise', 'std3', 'loStd3', 'hiStd3', 
'lotsize', 'optPrice', 'optMargin', 'rom']

optsList = [f for f in fs if f[-3:] == 'pkl']

df1 = pd.concat([pd.read_pickle(fspath+f) 
                 for f in optsList], axis=0, sort=True).reset_index(drop=True)[cols]

# filter for high probability
df2 = df1[((df1.strike > df1.hi52) | 
           (df1.strike < df1.lo52)) & 
          (df1.optPrice > minOptPrice) & 
          (df1.rom > minRom)]

# recalculated the prices and margins
with get_connected('nse', 'live') as ib:
    df2 = recalc_opts(ib, df2)

# group by the largest N return on margin
df2 = grp_opts(df2)
df3 = df2.groupby('symbol').apply(lambda x: x.nlargest(nLargest, 'rom'))

df4 = df3.assign(expPrice=[get_prec(p*1.1, 0.05) for p in df3.optPrice])
# df4 = df4.assign(remqty=[remqty[u] for u in df4.symbol])
# df4 = df4.assign(expQty=df4.remqty)

df5 = df4[df4.rom>0.15]

Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests


In [10]:
# make calls and puts watch - to quickly weed out risky options
gb = df5.groupby('right')

if 'C' in [k for k in gb.indices]:
#     df5_calls = gb.get_group('C').reset_index(drop=True).sort_values(['symbol', 'strike'], ascending=[True, True])
    df5_calls = gb.get_group('C').reset_index(drop=True)
    
    df5_callsymbols = df5_calls.symbol.unique()
    watchcalls = [('DES', s, 'STK', 'NSE') if s not in ['NIFTY50', 'BANKNIFTY'] else ('DES', s, 'IND', 'NSE') for s in df5_callsymbols]
    df5_wp = util.df(watchcalls)
    df5_wp.to_csv(fspath+'callswatch.csv', index=None, header=False)
    
else:
    df5_calls = pd.DataFrame([]) # empty dataframe

if 'P' in [k for k in gb.indices]:
#     df5_puts = gb.get_group('P').reset_index(drop=True).sort_values(['symbol', 'strike'], ascending=[True, False])
    df5_puts = gb.get_group('P').reset_index(drop=True)
    
    # make watchlist
    df5_putssymbols = df5_puts.symbol.unique()
    watchputs = [('DES', s, 'STK', 'NSE') if s not in ['NIFTY50', 'BANKNIFTY'] else ('DES', s, 'IND', 'NSE') for s in df5_putssymbols]
    df5_wp = util.df(watchputs)
    df5_wp.to_csv(fspath+'putswatch.csv', index=None, header=False)
else:
    df5_puts = pd.DataFrame([]) # empty dataframe

# output the consolidated puts and calls dataframe
df5 = pd.concat([df5_puts, df5_calls]).reset_index(drop=True)
df5.to_csv(fspath+'check.csv', index=None, header=True)

In [14]:
df5 = df5.assign(expQty=3)

In [20]:
df5.loc[df5.symbol == 'BSOFT', 'expPrice'] = 0.55

In [21]:
df5

optId     symbol right expiration  dte   ...    optPrice  optMargin       rom  expPrice  expQty
symbol                                                      ...                                                   
ACC       131  355720294        ACC     C   20190425    7   ...       2.000  123925.46  0.232398      2.20       3
          132  355720300        ACC     C   20190425    7   ...       1.700  123926.84  0.197536      1.85       3
          133  355949780        ACC     C   20190425    7   ...       1.475  123926.84  0.171391      1.60       3
ADANIENT  136  359177396   ADANIENT     C   20190425    7   ...       0.475  207591.98  0.329492      0.50       3
          134  359177381   ADANIENT     C   20190425    7   ...       0.325  207656.42  0.225372      0.35       3
          137  359177406   ADANIENT     C   20190425    7   ...       0.225  207656.42  0.156027      0.25       3
BAJFINANC 144  359223967  BAJFINANC     C   20190425    7   ...       2.600  137795.94  0.169816      2.85       3
BANKNIFTY 152  357896594  BANKNIFTY     C   20190425    7   ...      21.350   85090.81  0.180654     23.50       3
BSOFT     16   352126111      BSOFT     P   20190425    8   ...       0.300   51002.53  0.416891      0.55       3
GODFRYPHL 166  352319438  GODFRYPHL     C   20190425    8   ...       1.750  150591.14  0.256240      1.90       3
          167  352670730  GODFRYPHL     C   20190425    8   ...       1.050  150591.14  0.153744      1.15       3
ICICIPRUL 50   352138255  ICICIPRUL     P   20190425    8   ...       0.475  101041.74  0.222124      0.50       3
NESTLEIND 178  359174286  NESTLEIND     C   20190425    7   ...      20.250  100697.35  0.361976     22.30       3
          177  359174276  NESTLEIND     C   20190425    7   ...      15.525  100695.70  0.277519     17.10       3
          180  359174293  NESTLEIND     C   20190425    7   ...      11.700  100693.53  0.209149     12.85       3
RELIANCE  211  352136467   RELIANCE     C   20190425    7   ...       1.150  124184.17  0.166688      1.25       3
TCS       219  359232525        TCS     C   20190425    7   ...       3.250   97477.15  0.300070      3.60       3
          220  359232534        TCS     C   20190425    7   ...       2.250   97477.15  0.207741      2.50       3

[18 rows x 22 columns]

In [22]:
sum(df5.expQty*df5.expPrice*df5.lotsize)

50182.5

In [23]:
opt_contracts = [Contract(conId=c) for c in df5.optId]
with get_connected('nse', 'live') as ib:
    qopts = ib.qualifyContracts(*opt_contracts)

In [29]:
orders = [LimitOrder('SELL', qty*lotsize, price) for qty, lotsize, price in zip(df5.expQty, df5.lotsize, df5.expPrice)]

In [33]:
with get_connected('nse', 'live') as ib:
    limitTrade = [ib.placeOrder(c, o) for c, o in zip(qopts,  orders)]

Peer closed connection
clientId 1 already in use?
API connection failed: TimeoutError()


commissionReport: No execution found for CommissionReport(execId='00016710.5cb7e9a1.01.01', commission=25.18175, currency='INR')
Peer closed connection


In [26]:
# ib = get_connected('nse', 'live')

In [32]:
[ib.placeOrder(c, o) for c, o in zip(qopts,  orders)]

AssertionError: 

Error 103, reqId 13769: Duplicate order id


In [31]:
ib.reqGlobalCancel()